In [63]:
library(bnlearn)
library(Rgraphviz)
library(parallel)
library(tidyverse)
suppressWarnings(library(dplyr))
no_cores <- detectCores()-1
cl <- makeCluster(no_cores)
### dataset needs to be preprocessed because  for example had values as factors. My functions does not work w/factorslearning.test

Today i learnt that the k2 algorithm infers the dag, the directed acyclic graph, so the structure of the bayesian network from the dataset. Not from the fitted model. 
What i initially thought was that you happen to have a model and you want to reverse engineer that. 

In [64]:
df <- data.frame(x1=c(1,1,0,1,0,0,1,0,1,0), x2=c(0,1,0,1,0,1,1,0,1,0), x3=c(0,1,1,1,0,1,1,0,1,0))

In [65]:
node.values <- function(dataset, parents) {
    # retrieves the unique values of the nodes
    #return a list of vectors, 
    # each vector contains the unique values of the corresponding node
    output <- list()
    for (kk in seq_along(parents)) {
        node <- parents[kk]
        node.vals <- unique(dataset[[node]]) #it was levels()
        output[[kk]] <- node.vals
    }
    return(output)
}

old.alphaijk <- function(i, pii, dataset, nodes.order) {
    actual_node <- nodes.order[i]
    all.nodes <- c(pii, actual_node)
    subset <- dataset[all.nodes]
    unique.instantiations <- expand.grid(node.values(dataset, all.nodes))
    output <- c()
    for (tracker in 1:dim(unique.instantiations)[1]){
        aa <- 0
        for (j in 1:dim(subset)[1]) {
            ifelse(all(as.vector(unlist(subset[j,])) == as.vector(unlist(unique.instantiations[tracker,]))), aa <- aa +1, FALSE)
        }
        output <- c(output,aa)
    }
    return(output)
}

## OPtimized by copilot:
fastalphaijk <- function(i, pii, dataset, nodes.order) {
    actual_node <- nodes.order[i]
    all.nodes <- c(pii, actual_node)
    subset <- dataset[all.nodes]
    unique.instantiations <- expand.grid(node.values(dataset, all.nodes))
    colnames(unique.instantiations) <- colnames(subset)
    subset.counted <- subset |> group_by(across(everything())) |> summarise(n = n(), .groups="drop")
    checking <-  left_join(unique.instantiations, subset.counted, by=colnames(subset), na_matches="na") 
    checking[is.na(checking)]<- 0
    return(checking$n)
}


k2helper <- function(i, pii, dataset, nodes.order) {
    # the "f" function in the pdf
    phi_i <- expand.grid(node.values(dataset, pii)) 
    #cat("phi.i", phi_i)
    q_i <- dim(phi_i)[1] # works good
    #cat("qi.i", q_i)
    nodes <- colnames(dataset)
    #cat("nodes", nodes)
    i.th.node <- nodes[i]
    #cat("ithnode", i.th.node)
    v_i <- node.values(dataset, i.th.node)
    #cat("vi", c(v_i))
    r_i <- length(v_i[[1]])
    #cat("ri", r_i)
    individual_alphas <- fastalphaijk(i, pii, dataset, nodes.order) # works good
    aijk <- prod(factorial(individual_alphas)) ##
    numeratore <- factorial(r_i-1)
    nij <- individual_alphas
    print(nij)
    nij <- nij + r_i-1
    denominatore <- factorial(nij)
    frazione <- numeratore/prod(denominatore)
    #print(aijk)
    #print(numeratore)
    #print(frazione)
    return(prod(frazione)* aijk)
}

k2 <- function(dataset, nodes.order, upperbound) {
    nodes <- colnames(dataset)
    for (i in seq_along(nodes.order)) {
        actual_node <- nodes.order[i]
        cat("\n", i,"\n")
        pii <- c()
        p.old <- k2helper(i, pii,dataset, nodes.order)
        cat("\n ",p.old, "\n should be ", 1/2772)
        proceed <- TRUE
        roof <- 0
        while (proceed && length(pii) < upperbound) {
            predecessors_idx <- ifelse(i-1>=0, i-1, 0)
            z <- nodes.order[1:predecessors_idx]
            piiuz <- unique(pii,z)
            p.new <- k2helper(i, piiuz, dataset, nodes.order)
            roof <- roof+1
            if (p.new >= p.old) {
                p.old <- p.new
                pii <- piiuz
                ifelse(roof>length(nodes)^length(nodes), proceed <- FALSE, proceed <- TRUE)
            } else  {
                proceed <- FALSE
            }
        }
        cat('\nNode', nodes.order[i], '--> Parents:', pii,'\n') 
    }
    return(TRUE)
}

In [66]:
k2(dataset=df, upperbound=3, nodes.order=colnames(df))


 1 
[1] 5 5

  0.02777778 
 should be  0.0003607504[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5

Node x1 --> Parents: 

 2 
[1] 5 5

  0.02777778 
 should be  0.0003607504[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5
[1] 5 5

Node x2 --> Parents: 

 3 
[1] 4 6

  0.02857143 
 should be  0.0003607504[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6
[1] 4 6

Node x3 --> Parents: 


[1] TRUE